# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            #print('\n')
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion. 

### The event_datafile_new.csv contains the following columns: 
0. artist 
1. firstName of user
2. gender of user
3. item number in session
4. last name of user
5. length of the song
6. level (paid or free song)
7. location of the user
8. sessionId
9. song title
10. userId

Screenshot of the csv data: <font color=red>**event_datafile_new.csv**</font> <br>

<img src="images/image_event_datafile_new.jpg">

## Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect() # To establish connection and begin executing queries, need a session
except Exception as e:
    print(e)


## Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

## Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Query 1
### In order to query for `sessionId` and `itemInSession`, I use these columns to create a compound primary key. `SessionId` will be the *partition key* and `itemInSession` the *cluster key*.

In [8]:
query = "CREATE TABLE IF NOT EXISTS musicapp_history1"
query = query + "(sessionId int, itemInSession int, artist_name text, \
                song_title text, song_length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)             

## Insert data in tables

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## INSERT statements
        query = "INSERT INTO musicapp_history1 (sessionId, itemInSession, artist_name, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

## SELECT statement to verify that the data have been inserted into the table and query 1 returns the data
### Query 1 returns the artist, song title and song's length in the music app history that was heard during `sessionId` = 338, and `itemInSession` = 4

In [11]:
query = "SELECT artist_name, song_title, song_length FROM musicapp_history1 WHERE \
            sessionId=338 AND itemInSession=4"
try:
    df_query1 = pd.DataFrame( list(session.execute(query)) )
except Exception as e:
    print(e)

print(df_query1)


  artist_name                       song_title  song_length
0   Faithless  Music Matters (Mark Knight Dub)   495.307312


## Query 2
### We have to query for `userId`, `sessionId` and sort by `itemInSession`. Therefore, I create a new table and I use a compound primary key. Here, `userId` is the *partition key* and `sessionId` the *first cluster key*. In order to allow for sorting by `itemInSession`, this column will be the *second cluster key*.

In [12]:
query = "CREATE TABLE IF NOT EXISTS musicapp_history2"
query = query + "(userId int, sessionId int, itemInSession int, artist_name text, \
                song_title text, firstName text, lastName text, PRIMARY KEY (userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

## Insert data in table

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO musicapp_history2 (userId, sessionId, itemInSession, artist_name, \
                song_title, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

## SELECT to verify that the data have been inserted into the table
### Query 2 returns the name of artist, song and user in the music app history that was heard during `sessionId` = 338, and `itemInSession` = 4. The output will be sorted by `itemInSession`.

In [21]:
## SELECT statement to verify the data was entered into the table
query = "SELECT artist_name, song_title, firstName, lastName FROM musicapp_history2 WHERE \
            userId=10 AND sessionId=182 ORDER BY sessionId, itemInSession ASC;"
try:
    df_query2 = pd.DataFrame( list(session.execute(query)) )
except Exception as e:
    print(e)

print(df_query2)

         artist_name                                         song_title  \
0   Down To The Bone                                 Keep On Keepin' On   
1       Three Drives                                        Greece 2000   
2  Sebastien Tellier                                          Kilometer   
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...   

  firstname lastname  
0    Sylvie     Cruz  
1    Sylvie     Cruz  
2    Sylvie     Cruz  
3    Sylvie     Cruz  


## QUERY 3
### Search for all user names (first and last) in the music app history who listened to the song 'All Hands Against His Own'. I created a new table to support this query. This time the compound primary key has the *partition key* `song_title` and the *cluster key* `userId. The cluster key will allow to return *all* users.

In [22]:
query = "CREATE TABLE IF NOT EXISTS musicapp_history3"
query = query + "(song_title text, userId int, firstName text, lastName text, PRIMARY KEY (song_title, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

## Insert data in table

In [23]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO musicapp_history3 (userId, song_title, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (int(line[10]),  line[9], line[1], line[4]))

## SELECT statement to verify that the data have been inserted into the table
### Query 3 returns the user name (first and last) from the music app history who listened to the song 'All Hands Against His Own'. The output will return all user because we defined `userId` as cluster key.

In [25]:
query = "SELECT firstName, lastName FROM musicapp_history3 WHERE \
            song_title='All Hands Against His Own';"
try:
    df_query3 = pd.DataFrame( list(session.execute(query)) )
except Exception as e:
    print(e)

print(df_query3)

    firstname lastname
0  Jacqueline    Lynch
1       Tegan   Levine
2        Sara  Johnson


### Drop the tables before closing out the sessions

In [26]:
query = "DROP TABLE IF EXISTS musicapp_history1"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE IF EXISTS musicapp_history2"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "DROP TABLE IF EXISTS musicapp_history3"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()